# Imports

In [31]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.fire_data import *
from src.data.read_data import *
from src.gen_functions import *
import seaborn as sns
output_notebook()
# set font size 
plt.rcParams.update({'font.size': 16})

from datetime import timedelta
from scipy.stats import spearmanr
from matplotlib.ticker import MaxNLocator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Loading BokehJS ...

In [5]:
# transition from acceptable to unhealthy for sensitive group and to unhealthy. 
transition_dict = { 'PM2.5': [0, 35.5, 55.4, 1e3],
                  'PM10': [0, 155, 254, 1e3],
                  'O3':[0, 70 , 85, 1e3],
                  'SO2':[0, 75, 185, 1e3],
                  'NO2': [0, 100, 360, 1e3],
                  'CO': [0, 6.4, 12.5, 1e3]}

gas_list = ['PM2.5', 'PM10', 'O3', 'CO', 'NO2', 'SO2']

def add_is_holiday(df,th_holiday):
    """ add is_holiday columns. df must have 'datetime' columns
    
    """
    # prepare datetime columns
    try: 
        
        df['datetime'] = pd.to_datetime(df['datetime'])
    
    except: 
        df = df.reset_index() 
        df['datetime'] = pd.to_datetime(df['datetime'])
        
    df['date'] = pd.to_datetime(df['datetime'].dt.date)
    th_holiday['date'] = pd.to_datetime(th_holiday['date'])
    df = df.merge(th_holiday[['date','name']], on='date',how='left')
    df['is_holiday'] = ~df['name'].isna()*1
    return df.drop('name',axis=1)

def add_season(df,start_month='-12-01', end_month='-04-30'):
    # add winter season column 
    # df.index must be datetime format sorted in ascending order
    df = df.sort_index()
    df['year'] = df.index.year 
    df['season'] = 'other'
    for year in df.year.unique():
        start_date = str(year)+ start_month
        end_date = str(year+1)+ end_month
        label = 'winter_'+str(year)
    
        df.loc[start_date:end_date,'season'] = label 
        
    return df

In [3]:
b_folder='../data/pm25/'
a4th_folder ='../data/air4thai_hourly/'
cm_folder ='../data/cm_proc/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'

In [10]:
aqm1 = pd.read_csv(cm_folder + '35t.csv').set_index('datetime').dropna(how='all')
aqm1.index = pd.to_datetime(aqm1.index)
aqm2 = pd.read_csv(cm_folder + '36t.csv').set_index('datetime').dropna(how='all')
aqm2.index = pd.to_datetime(aqm2.index)
print(aqm2.columns)
# keep only the data after the satallite data which is 200-11-11 13 am
aqm2_01 = aqm2[aqm2.index>='2000-11-01 00:00:00'].copy()
aqm2_01 = add_season(aqm2_01)
print(aqm2_01.shape)

# weather data 
filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/Mueang_Chiang_Mai.csv'
wea = pd.read_csv(filename)
wea.drop(['Time','Dew Point(C)','Wind Gust(kmph)','Pressure(in)','Condition','Precip.(in)'], axis=1, inplace=True)
wea['datetime'] = pd.to_datetime(wea['datetime'])
# merge with weather 

aqm2_01 = aqm2_01.merge(wea, left_index=True, right_on ='datetime',how='inner').set_index('datetime')
print(aqm2_01.shape)

Index(['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5'], dtype='object')
(167708, 8)
(164830, 12)


In [12]:
city_info = {'Country': 'Thailand',
 'City': 'Chiang Mai',
 'City (ASCII)': 'Chiang Mai',
 'Region': 'Chiang Mai',
 'Region (ASCII)': 'Chiang Mai',
 'Population': '200952',
 'Latitude': '18.7904',
 'Longitude': '98.9847',
 'Time Zone': 'Asia/Bangkok'}

x = merc_x(city_info['Longitude'])
y = merc_y(city_info['Latitude'])
stepx = 800E3
stepy = stepx

#fire_v = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv')
m_filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m.csv'
fire = pd.read_csv(m_filename, dtype={'version': str})
columns_to_drop = ['acq_date','satellite','instrument','version','daynight','bright_t31','type']
fire = fire.drop(columns_to_drop,axis=1)
fire['datetime'] = pd.to_datetime(fire['datetime'])
fire = fire.sort_values('datetime')
fire = fire.set_index('datetime')
# remove the data before '2002-07-04' because there is only one satellite
fire = fire.loc['2002-07-04':]
print(fire.shape)

fire = add_season(fire)

# add distance columns
fire['distance'] = np.sqrt((fire['lat_km'] - y/1000) **2 + ((fire['long_km'] - x/1000)**2))
fire.head()

(3779468, 10)


,latitude,longitude,brightness,scan,track,acq_time,confidence,frp,lat_km,long_km,year,season,distance
datetime,,,,,,,,,,,,,
2002-07-04 13:03:00,14.4216,102.8814,315.1,2.8,1.6,603,67,27.0,1612.0,11453.0,2002,other,665.551109
2002-07-04 13:04:00,19.8946,105.1144,312.1,1.5,1.2,604,52,11.4,2246.0,11701.0,2002,other,694.254989
2002-07-04 13:04:00,20.3829,104.9449,312.6,1.5,1.2,604,48,7.5,2304.0,11682.0,2002,other,689.069015
2002-07-04 13:04:00,20.3938,104.9437,342.2,1.5,1.2,604,91,73.5,2305.0,11682.0,2002,other,689.341767
2002-07-04 13:05:00,22.1193,104.6895,317.5,1.5,1.2,605,71,14.1,2510.0,11654.0,2002,other,747.092082


In [17]:
# create power column and drop unncessary columns
fire['power'] = fire['scan']*fire['track']*fire['frp']
fire['count'] = 1
fire.drop(['latitude', 'longitude', 'brightness','distance',
       'power', 'count'], axis=1)
fire.head()

,latitude,longitude,brightness,scan,track,acq_time,confidence,frp,lat_km,long_km,year,season,distance,power,count
datetime,,,,,,,,,,,,,,,
2002-07-04 13:03:00,14.4216,102.8814,315.1,2.8,1.6,603,67,27.0,1612.0,11453.0,2002,other,665.551109,120.96,1
2002-07-04 13:04:00,19.8946,105.1144,312.1,1.5,1.2,604,52,11.4,2246.0,11701.0,2002,other,694.254989,20.52,1
2002-07-04 13:04:00,20.3829,104.9449,312.6,1.5,1.2,604,48,7.5,2304.0,11682.0,2002,other,689.069015,13.50,1
2002-07-04 13:04:00,20.3938,104.9437,342.2,1.5,1.2,604,91,73.5,2305.0,11682.0,2002,other,689.341767,132.30,1
2002-07-04 13:05:00,22.1193,104.6895,317.5,1.5,1.2,605,71,14.1,2510.0,11654.0,2002,other,747.092082,25.38,1


# Model PM2.5

In [26]:
cols = ['PM2.5', 'Temperature(C)', 'Humidity(%)', 'Wind', 'Wind Speed(kmph)']
pm25 = aqm2_01[cols].dropna()
# keep the data since '2011-05-07'
pm25 = pm25.loc['2010-01-01':] 

In [45]:
pm25['Wind'] = pm25['Wind'].astype('category')
pm25_dummies = pd.get_dummies(pm25['Wind'])
pm25_dummies

,CALM,E,ENE,ESE,N,NE,NNE,NNW,NW,S,SE,SSE,SSW,SW,VAR,W,WNW,WSW
datetime,,,,,,,,,,,,,,,,,,
2011-05-17 16:00:00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2011-05-17 17:00:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2011-05-17 18:00:00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2011-05-17 19:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2011-05-17 20:00:00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-10 10:00:00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2020-05-10 11:00:00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2020-05-10 12:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [59]:
# one hot encode data 
pm25['Wind'] = pm25['Wind'].astype('category')
dummies = pd.get_dummies(pm25['Wind'])

# group the wind direction into major wind direction 
direction_to_collpse = ['ENE', 'ESE', 'NE', 'NNE', 'NNW', 'NW', 'SE', 'SSE', 'SSW', 'SW',  'WNW', 'WSW']

for direction in direction_to_collpse:
    if len(direction)>1:
        for char in set(direction):
            dummies[char] = dummies[char] + dummies[direction]
        dummies.drop(direction, axis=1,inplace=True)

# group the ''
major_direction = ['E','N','S','W']
for direction in major_direction:
    dummies[direction] = dummies[direction] + dummies['VAR']


In [60]:
dummies

,CALM,E,N,S,VAR,W
datetime,,,,,,
2011-05-17 16:00:00,0,1,1,0,0,0
2011-05-17 17:00:00,0,1,0,0,0,0
2011-05-17 18:00:00,0,0,0,1,0,1
2011-05-17 19:00:00,0,0,0,1,0,1
2011-05-17 20:00:00,0,0,0,1,0,1
...,...,...,...,...,...,...
2020-05-10 10:00:00,0,0,0,1,0,1
2020-05-10 11:00:00,0,1,0,1,0,0
2020-05-10 12:00:00,0,0,0,0,1,0


In [53]:
dummies[dummies>1].dropna(how='all')

,CALM,E,ENE,ESE,N,NE,NNE,NNW,NW,S,SE,SSE,SSW,SW,VAR,W,WNW,WSW
datetime,,,,,,,,,,,,,,,,,,
2011-05-17 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2011-05-17 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2011-05-17 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2011-05-17 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2011-05-17 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-10 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-10 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
2020-05-10 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN


In [56]:
dummies = pd.get_dummies(pm25['Wind'])

dummies.loc['2011-05-17 22:00:00']

CALM    0
E       0
ENE     0
ESE     0
N       0
NE      0
NNE     0
NNW     0
NW      0
S       1
SE      0
SSE     0
SSW     0
SW      0
VAR     0
W       0
WNW     0
WSW     0
Name: 2011-05-17 22:00:00, dtype: uint8